In [ ]:
import pandas as pd
import numpy as np

dir_path = './Amazon_Review/Books/'
rating_file = 'ratings_Books.csv'

In [ ]:
def read_user_rating_records():
    col_names = ['user_id', 'item_id', 'rating', 'timestamp']
    data_records = pd.read_csv(dir_path + rating_file, sep=',', names=col_names, engine='python')
    return data_records

data_records = read_user_rating_records()
data_records.head()
data_records.iloc[[1, 10, 20]]

In [ ]:
print(len(data_records['user_id'].value_counts()), len(data_records['item_id'].value_counts()))

In [ ]:
data_records.loc[data_records.rating < 4, 'rating'] = 0
data_records.loc[data_records.rating >= 4, 'rating'] = 1
data_records = data_records[data_records.rating > 0]
print(len(data_records['user_id'].unique()), len(data_records['item_id'].unique()))

In [ ]:
from copy import deepcopy
def remove_infrequent_items(data, min_counts=5):
    df = deepcopy(data)
    counts = df['item_id'].value_counts()
    df = df[df["item_id"].isin(counts[counts >= min_counts].index)]

    print("items with < {} interactoins are removed".format(min_counts))
    # print(df.describe())
    return df

def remove_infrequent_users(data, min_counts=10):
    df = deepcopy(data)
    counts = df['user_id'].value_counts()
    df = df[df["user_id"].isin(counts[counts >= min_counts].index)]

    print("users with < {} interactoins are removed".format(min_counts))
    # print(df.describe())
    return df

filtered_data = remove_infrequent_users(data_records, 20)
filtered_data = remove_infrequent_items(filtered_data, 20)
print('num of users:{}, num of items:{}'.format(len(filtered_data['user_id'].unique()), len(filtered_data['item_id'].unique())))

In [ ]:
item_list = filtered_data['item_id'].unique()
item_set = set(item_list)

print(item_list[:10])

review_file = 'reviews_Books_5.json.gz'

import json
import gzip

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)
        # yield json.dumps(eval(l))

review_dict = dict()  # [review_id] = review_text
review_helpful = dict()
for l in parse(dir_path + review_file):
    if l['asin'] in item_set:
        if l['asin'] in review_dict:
            if l['helpful'][0] / float(l['helpful'][1] + 0.01) > review_helpful[l['asin']] and len(l['reviewText']) > 10:
                review_dict[l['asin']] = l['reviewText']
                review_helpful[l['asin']] = l['helpful'][0] / float(l['helpful'][1] + 0.01)
        else:
            if len(l['reviewText']) > 10:
                review_dict[l['asin']] = l['reviewText']
                review_helpful[l['asin']] = l['helpful'][0] / float(l['helpful'][1] + 0.01)

In [ ]:
item_without_review = []
for item_id in item_list:
    if item_id not in review_dict:
        item_without_review.append(item_id)

print(item_without_review)

for item_id in item_without_review:
    filtered_data = filtered_data[filtered_data['item_id'] != item_id]

item_list = filtered_data['item_id'].unique()
print(len(item_list))

In [ ]:
for item_id, review in review_dict.items():
    if len(review) < 5:
        print(item_id)

In [ ]:
def convert_data(data):
    # for each user, sort by timestamps
    df = deepcopy(data)
    df_ordered = df.sort_values(['timestamp'], ascending=True)
    data = df_ordered.groupby('user_id')['item_id'].apply(list)
    #print(data)
    #time_l = df_ordered.groupby('user')['checkin_time'].apply(list)
    #print(time_l)
    print("succressfully created sequencial data! head:", data.head(5))
    unique_data = df_ordered.groupby('user_id')['item_id'].nunique()
    data = data[unique_data[unique_data >= 10].index]
    print(data[:10])
    print(len(data))
    return data

seq_data = convert_data(filtered_data)
print(type(seq_data))

In [ ]:
user_item_dict = seq_data.to_dict()
user_mapping = []
item_set = set()
for user_id, item_list in seq_data.iteritems():
    user_mapping.append(user_id)
    for item_id in item_list:
        item_set.add(item_id)
item_mapping = list(item_set)

print(len(user_mapping), len(item_mapping))

In [ ]:
def generate_inverse_mapping(data_list):
    inverse_mapping = dict()
    for inner_id, true_id in enumerate(data_list):
        inverse_mapping[true_id] = inner_id
    return inverse_mapping

def convert_to_inner_index(user_records, user_mapping, item_mapping):
    inner_user_records = []
    user_inverse_mapping = generate_inverse_mapping(user_mapping)
    item_inverse_mapping = generate_inverse_mapping(item_mapping)

    for user_id in range(len(user_mapping)):
        real_user_id = user_mapping[user_id]
        item_list = list(user_records[real_user_id])
        for index, real_item_id in enumerate(item_list):
            item_list[index] = item_inverse_mapping[real_item_id]
        inner_user_records.append(item_list)

    return inner_user_records, user_inverse_mapping, item_inverse_mapping

inner_data_records, user_inverse_mapping, item_inverse_mapping = convert_to_inner_index(user_item_dict, user_mapping, item_mapping)
print(inner_data_records[:5])

In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)
        

In [ ]:
save_obj(inner_data_records, 'Books_item_sequences')
save_obj(user_mapping, 'Books_user_mapping')
save_obj(item_mapping, 'Books_item_mapping')

In [ ]:
from scipy.sparse import csr_matrix

def generate_rating_matrix(train_set, num_users, num_items):
    # three lists are used to construct sparse matrix
    row = []
    col = []
    data = []
    for user_id, article_list in enumerate(train_set):
        for article in article_list:
            row.append(user_id)
            col.append(article)
            data.append(1)

    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

    return rating_matrix

rating_matrix = generate_rating_matrix(inner_data_records, len(user_mapping), len(item_mapping))
rating_matrix = rating_matrix.transpose()

from sklearn.metrics.pairwise import cosine_similarity

relation_matrix = cosine_similarity(rating_matrix)

In [ ]:

rating_matrix.nnz / float(len(user_mapping) * len(item_mapping))

In [ ]:
np.fill_diagonal(relation_matrix, 0)
max_count = 0
for i in range(len(item_mapping)):
    max_count = max(np.count_nonzero((relation_matrix[i] >= 0.2) == True), max_count)
    
print max_count

count = 0
for i in range(len(item_mapping)):
    if np.count_nonzero((relation_matrix[i] >= 0.2) == True) > 0:
        count += 1

print count
print np.max(relation_matrix)
print relation_matrix[0]
print relation_matrix[-1]

In [ ]:
relation_matrix[relation_matrix < 0.2] = 0
relation_matrix[relation_matrix > 0] = 1
relation_matrix = csr_matrix(relation_matrix)
print len(user_mapping), len(item_mapping)

In [ ]:
import re
import nltk

# generate the whole document
all_review = []
for item_id in item_mapping:
    all_review.append([review_dict[item_id]])

# use nltk to remove stopwords, and stemming each word
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
porter_stemmer = nltk.PorterStemmer()

review_str = []
for i, movie in enumerate(all_review):
    # Use regular expressions to do a find-and-replace
    letters_only = re.sub("[^a-zA-Z]",  # The pattern to search for
                          " ",  # The pattern to replace it with
                          movie[0])  # The text to search
    # print letters_only

    letters_only = letters_only.lower()
    tokens = nltk.word_tokenize(letters_only)

    tokens = [w for w in tokens if w.lower() not in stopwords_set]
    # print tokens

    porter = [porter_stemmer.stem(t) for t in tokens]
    # print porter
    all_review[i] = porter
    review_str.append(' '.join(porter))
    
print review_str[-1]   

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, stop_words=None, min_df=3)
word_counts = vectorizer.fit_transform(review_str)
vocab = vectorizer.get_feature_names()

print len(vocab)
print word_counts.data.max()
print word_counts.data.min()
print len(item_mapping)

In [ ]:

rating_matrix.nnz / float(len(user_mapping) * len(item_mapping))

In [ ]:
def vocabulary_to_file(vocab):
    f0 = open('vocabulary.txt', 'w')

    for word in vocab:
        f0.write(word + '\n')
    f0.close()


def word_count_to_file(item_list, word_count):
    f0 = open('word_counts.txt', 'w')
    for i, document in enumerate(word_count):
        indices = document.indices
        counts = document.data
        num_words = document.count_nonzero()

        f0.write(str(item_list[i]) + ' ' + str(num_words))
        for j, indice in enumerate(indices):
            f0.write(' ' + str(indice) + ':' + str(counts[j]))
        f0.write('\n')
    f0.close()

vocabulary_to_file(vocab)
word_count_to_file(item_mapping, word_counts)

In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

In [ ]:
save_obj(user_item_dict, 'Books_user_records')
save_obj(user_mapping, 'Books_user_mapping')
save_obj(item_mapping, 'Books_item_mapping')
save_obj(relation_matrix, 'item_relation')

In [ ]:
print vocab[:10]
print all_review[-1]
print review_str[-1]

In [ ]:
word_to_index = dict()
for w_id, word in enumerate(vocab):
    word_to_index[word] = w_id

all_review_index = []
for i in range(len(review_str)):
    cur_review = review_str[i].split(' ')
    cur_index = []
    for word in cur_review:
        if word in word_to_index:
            cur_index.append(word_to_index[word])
    all_review_index.append(cur_index)
    
print all_review_index[-1]

In [ ]:
save_obj(all_review_index, 'review_word_sequence')

In [ ]:
print seq_data[-1]
user_inverse_mapping = generate_inverse_mapping(user_mapping)
item_inverse_mapping = generate_inverse_mapping(item_mapping)
print user_item_dict[user_mapping[-1]]
tmp = []
for item_id in seq_data[-1]:
    tmp.append(item_inverse_mapping[item_id])
print sorted(tmp)

In [ ]:
print all_review[-1]

In [ ]:
for i in range(word_counts.shape[0]):
    if word_counts.getrow(i).getnnz() == 0:
        print i